In [56]:
import sys
sys.path.append('C:/Users/pc/Documents/GitHub/Private_Project/personal_project')
from collections import defaultdict
from datetime import datetime, timedelta
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import pickle
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
from functools import partial
# [Default]
def OS_Driver(os,browser):
    if os.lower() == 'windows':
        if browser.lower() == 'phantom':
            driver = webdriver.PhantomJS('C:/Users/pc/Documents/phantomjs-2.1.1-window/bin/phantomjs.exe')
        else:
            driver = webdriver.Chrome('C:/Users/pc/Documents/chromedriver.exe')
    elif os.lower() == 'mac':
        if browser.lower() == 'phantom':
            driver = webdriver.PhantomJS(
                '/Users/hyunyoun/Documents/GitHub/Private_Project/phantomjs-2.1.1/bin/phantomjs')
        else:
            driver = webdriver.Chrome('/Users/hyunyoun/Documents/GitHub/Private_Project/chromedriver')
    return driver

# Use naver api
def OpenAPI(apiFile):
    apiKey = pickle.load(open(apiFile, 'rb'))
    return apiKey

## Define Error
class NotMatch(Exception):
    def __init__(self, f):
        self.f = f
    def __str__(self):
        return self.f
    
## Setting BeautifulSoup
def Setting_BeautifulSoup(parser):
    if parser.lower() == 'xml':
        parser = 'lxml'
    elif parser.lower() == 'html':
        parser = 'html.parser'
    else:
        raise NotMatch('Not Match parser')
    return parser

## Using BeautifulSoup
def WebPageUsingBS(url, htmltype, openApi=None):
    htmltype = Setting_BeautifulSoup(htmltype)
    if openApi == None:
        response = requests.get(url)
    else:
        response = requests.get(url, headers=openApi)
    soup = BeautifulSoup(response.content, htmltype)
    return soup

def Realtime_Daum(url):
    searchTime = datetime.now()
    dateIs = searchTime.strftime('%Y-%m-%d')
    timeIs = searchTime.strftime('%H:%M:%S')
    soup = WebPageUsingBS(url, 'html')
    realtimeIssue = soup.find('div', class_='aside_search')
    tit_news = realtimeIssue.find('h3', class_='tit_news').text
    cont_aside = realtimeIssue.find('ul', class_='tab_aside').find_all('div',class_='cont_aside')
    return dateIs, timeIs, cont_aside

def SearchNewstopicForDaum(date, time, soup_ul_div):
    category = soup_ul_div.find('strong', class_ = 'screen_out').text
    list_ranking = list(map(lambda x: (date, time, 'daum', category, x.find('em').text, x.find('a').text), soup_ul_div.find_all('li')))
    rank_Df = pd.DataFrame(list_ranking)
    rank_Df.rename({0:'date', 1:'time', 2:'site', 3:'category', 4:'rank', 5:'topic'}, inplace = True, axis = 1)
    return rank_Df

def Main_Daum():
    url = Resource()[0]
    realTime = Realtime_Daum(url)
    out_Dfs = pd.DataFrame(columns=['date','time','site','category','rank','topic'])
    for categoryTopic in realTime[2]:
        newsTopics = SearchNewstopicForDaum(realTime[0], realTime[1], categoryTopic)
        if len(out_Dfs) == 0:
            out_Dfs = newsTopics
        else:
            out_Dfs = pd.concat([out_Dfs, newsTopics], axis = 0)
    out_Dfs.reset_index(drop=True, inplace=True)
    return out_Dfs

def Realtime_Naver(url):
    searchTime = datetime.now()
    dateIs = searchTime.strftime('%Y-%m-%d')
    timeIs = searchTime.strftime('%H:%M:%S')
    soup = WebPageUsingBS(url, 'html')
    realtimeIssue = soup.find('td', class_='aside').find('div', class_='hottopic')
    categoryIssues = realtimeIssue.find_all('ol', class_= 'type15')
    return dateIs, timeIs, categoryIssues

def Category_Topic(x):
    if x == 'newstopic_news':
        y = r'뉴스'
    else:
        y = r'연예/스포츠'
    return y

def SearchNewstopicForNaver(date, time, soup_ul_div):
    category = Category_Topic(soup_ul_div.attrs['id'])
    list_ranking = list(map(lambda x: (date, time, 'naver', category, x.find('em').text, x.find('a')['title']), soup_ul_div.find_all('li')))
    rank_Df = pd.DataFrame(list_ranking)
    rank_Df.rename({0:'date', 1:'time', 2:'site', 3:'category', 4:'rank', 5:'topic'}, inplace = True, axis = 1)
    return rank_Df

def Main_Naver():
    url = Resource()[1]
    realTime = Realtime_Naver(url)
    out_Dfs = pd.DataFrame(columns=['date','time','site','category','rank','topic'])
    for categoryTopic in realTime[2]:
        newsTopics = SearchNewstopicForNaver(realTime[0], realTime[1], categoryTopic)
        if len(out_Dfs) == 0:
            out_Dfs = newsTopics
        else:
            out_Dfs = pd.concat([out_Dfs, newsTopics], axis = 0)
    out_Dfs.reset_index(drop=True, inplace=True)
    return out_Dfs

def Resource():
    daumWebpath = 'http://media.daum.net/ranking/popular/'
    naverWebpath = 'http://news.naver.com/main/ranking/popularDay.nhn?'
    return daumWebpath, naverWebpath

if __name__ == '__main__':
    pass

In [56]:
import sys
sys.path.append('C:/Users/pc/Documents/GitHub/Private_Project/personal_project')
from collections import defaultdict
from datetime import datetime, timedelta
import time
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import pickle
import chat_bot as cb
import Database_Handler as dh
from multiprocessing import Pool
from functools import partial
# [Default]
def OS_Driver(os,browser):
    if os.lower() == 'windows':
        if browser.lower() == 'phantom':
            driver = webdriver.PhantomJS('C:/Users/pc/Documents/phantomjs-2.1.1-window/bin/phantomjs.exe')
        else:
            driver = webdriver.Chrome('C:/Users/pc/Documents/chromedriver.exe')
    elif os.lower() == 'mac':
        if browser.lower() == 'phantom':
            driver = webdriver.PhantomJS(
                '/Users/hyunyoun/Documents/GitHub/Private_Project/phantomjs-2.1.1/bin/phantomjs')
        else:
            driver = webdriver.Chrome('/Users/hyunyoun/Documents/GitHub/Private_Project/chromedriver')
    return driver

# Use naver api
def OpenAPI(apiFile):
    apiKey = pickle.load(open(apiFile, 'rb'))
    return apiKey

## Define Error
class NotMatch(Exception):
    def __init__(self, f):
        self.f = f
    def __str__(self):
        return self.f
    
## Setting BeautifulSoup
def Setting_BeautifulSoup(parser):
    if parser.lower() == 'xml':
        parser = 'lxml'
    elif parser.lower() == 'html':
        parser = 'html.parser'
    else:
        raise NotMatch('Not Match parser')
    return parser

## Using BeautifulSoup
def WebPageUsingBS(url, htmltype, openApi=None):
    htmltype = Setting_BeautifulSoup(htmltype)
    if openApi == None:
        response = requests.get(url)
    else:
        response = requests.get(url, headers=openApi)
    soup = BeautifulSoup(response.content, htmltype)
    return soup

def Realtime_Daum(url):
    searchTime = datetime.now()
    dateIs = searchTime.strftime('%Y-%m-%d')
    timeIs = searchTime.strftime('%H:%M:%S')
    soup = WebPageUsingBS(url, 'html')
    realtimeIssue = soup.find('div', class_='aside_search')
    tit_news = realtimeIssue.find('h3', class_='tit_news').text
    cont_aside = realtimeIssue.find('ul', class_='tab_aside').find_all('div',class_='cont_aside')
    return dateIs, timeIs, cont_aside

def SearchNewstopicForDaum(date, time, soup_ul_div):
    category = soup_ul_div.find('strong', class_ = 'screen_out').text
    list_ranking = list(map(lambda x: (date, time, 'daum', category, x.find('em').text, x.find('a').text), soup_ul_div.find_all('li')))
    rank_Df = pd.DataFrame(list_ranking)
    rank_Df.rename({0:'date', 1:'time', 2:'site', 3:'category', 4:'rank', 5:'topic'}, inplace = True, axis = 1)
    return rank_Df

def Main_Daum():
    url = Resource()[0]
    realTime = Realtime_Daum(url)
    out_Dfs = pd.DataFrame(columns=['date','time','site','category','rank','topic'])
    for categoryTopic in realTime[2]:
        newsTopics = SearchNewstopicForDaum(realTime[0], realTime[1], categoryTopic)
        if len(out_Dfs) == 0:
            out_Dfs = newsTopics
        else:
            out_Dfs = pd.concat([out_Dfs, newsTopics], axis = 0)
    out_Dfs.reset_index(drop=True, inplace=True)
    return out_Dfs

def Realtime_Naver(url):
    searchTime = datetime.now()
    dateIs = searchTime.strftime('%Y-%m-%d')
    timeIs = searchTime.strftime('%H:%M:%S')
    soup = WebPageUsingBS(url, 'html')
    realtimeIssue = soup.find('td', class_='aside').find('div', class_='hottopic')
    categoryIssues = realtimeIssue.find_all('ol', class_= 'type15')
    return dateIs, timeIs, categoryIssues

def Category_Topic(x):
    if x == 'newstopic_news':
        y = r'뉴스'
    else:
        y = r'연예/스포츠'
    return y

def SearchNewstopicForNaver(date, time, soup_ul_div):
    category = Category_Topic(soup_ul_div.attrs['id'])
    list_ranking = list(map(lambda x: (date, time, 'naver', category, x.find('em').text, x.find('a')['title']), soup_ul_div.find_all('li')))
    rank_Df = pd.DataFrame(list_ranking)
    rank_Df.rename({0:'date', 1:'time', 2:'site', 3:'category', 4:'rank', 5:'topic'}, inplace = True, axis = 1)
    return rank_Df

def Main_Naver():
    url = Resource()[1]
    realTime = Realtime_Naver(url)
    out_Dfs = pd.DataFrame(columns=['date','time','site','category','rank','topic'])
    for categoryTopic in realTime[2]:
        newsTopics = SearchNewstopicForNaver(realTime[0], realTime[1], categoryTopic)
        if len(out_Dfs) == 0:
            out_Dfs = newsTopics
        else:
            out_Dfs = pd.concat([out_Dfs, newsTopics], axis = 0)
    out_Dfs.reset_index(drop=True, inplace=True)
    return out_Dfs

def Resource():
    daumWebpath = 'http://media.daum.net/ranking/popular/'
    naverWebpath = 'http://news.naver.com/main/ranking/popularDay.nhn?'
    return daumWebpath, naverWebpath

if __name__ == '__main__':
    pass

In [57]:
Main_Daum()

         date      time  site category rank   topic
0  2017-12-09  05:29:03  daum       전체    1     박주원
1  2017-12-09  05:29:03  daum       전체    2  모두의 연애
2  2017-12-09  05:29:03  daum       전체    3     배현진
3  2017-12-09  05:29:03  daum       전체    4     이민아
4  2017-12-09  05:29:03  daum       전체    5   포항 지진
5  2017-12-09  05:29:03  daum       전체    6      성소
6  2017-12-09  05:29:03  daum       전체    7    오늘날씨
         date      time  site category rank     topic
0  2017-12-09  05:29:03  daum       뉴스    1  박주원 국민의당
1  2017-12-09  05:29:03  daum       뉴스    2  김수지 아나운서
2  2017-12-09  05:29:03  daum       뉴스    3    한정우 기자
3  2017-12-09  05:29:03  daum       뉴스    4  배현진 아나운서
4  2017-12-09  05:29:03  daum       뉴스    5    이상현 앵커
5  2017-12-09  05:29:03  daum       뉴스    6    mbc 뉴스
6  2017-12-09  05:29:03  daum       뉴스    7     경찰 인사
         date      time  site category rank           topic
0  2017-12-09  05:29:03  daum       연예    1  발레교습소 백조클럽 손연재
1  2017-12-09  05:29:03  daum   

,date,time,site,category,rank,topic
0,2017-12-09,05:29:03,daum,전체,1,박주원
1,2017-12-09,05:29:03,daum,전체,2,모두의 연애
2,2017-12-09,05:29:03,daum,전체,3,배현진
3,2017-12-09,05:29:03,daum,전체,4,이민아
4,2017-12-09,05:29:03,daum,전체,5,포항 지진
5,2017-12-09,05:29:03,daum,전체,6,성소
6,2017-12-09,05:29:03,daum,전체,7,오늘날씨
7,2017-12-09,05:29:03,daum,뉴스,1,박주원 국민의당
8,2017-12-09,05:29:03,daum,뉴스,2,김수지 아나운서
9,2017-12-09,05:29:03,daum,뉴스,3,한정우 기자


In [ ]:
realTime = Realtime_Naver(url)
out_Dfs = pd.DataFrame(columns=['date','time','site','category','rank','topic'])
for categoryTopic in realTime[2]:
    newsTopics = SearchNewstopicForNaver(realTime[0], realTime[1], categoryTopic)
        if len(out_Dfs) == 0:
            out_Dfs = newsTopics
        else:
            out_Dfs = pd.concat([out_Dfs, newsTopics], axis = 0)
out_Dfs.reset_index(drop=True, inplace=True)

In [42]:
Main_Naver()

         date      time   site category rank              topic
0  2017-12-09  05:21:46  naver       뉴스    1           박주원 법적대응
1  2017-12-09  05:21:46  naver       뉴스    2   틸러슨 美대사관 예루살렘 이전
2  2017-12-09  05:21:46  naver       뉴스    3           서울대병원 파업
3  2017-12-09  05:21:46  naver       뉴스    4     브렉시트 1단계 협상 타결
4  2017-12-09  05:21:46  naver       뉴스    5          비결핵항산균 검출
5  2017-12-09  05:21:46  naver       뉴스    6            가상화폐 규제
6  2017-12-09  05:21:46  naver       뉴스    7           뉴스데스크 하차
7  2017-12-09  05:21:46  naver       뉴스    8       일요일 전국 눈이나 비
8  2017-12-09  05:21:46  naver       뉴스    9  전작권 전환 조건 조속히 갖춰야
9  2017-12-09  05:21:46  naver       뉴스   10         예루살렘 수도 인정
         date      time   site category rank        topic
0  2017-12-09  05:21:46  naver   연예/스포츠    1   모두의 연애 변우석
1  2017-12-09  05:21:46  naver   연예/스포츠    2  나 혼자 산다 전현무
2  2017-12-09  05:21:46  naver   연예/스포츠    3     그것이 알고싶다
3  2017-12-09  05:21:46  naver   연예/스포츠    4     백조클럽 손연재
4  201

,date,time,site,category,rank,topic
0,2017-12-09,05:21:46,naver,뉴스,1,박주원 법적대응
1,2017-12-09,05:21:46,naver,뉴스,2,틸러슨 美대사관 예루살렘 이전
2,2017-12-09,05:21:46,naver,뉴스,3,서울대병원 파업
3,2017-12-09,05:21:46,naver,뉴스,4,브렉시트 1단계 협상 타결
4,2017-12-09,05:21:46,naver,뉴스,5,비결핵항산균 검출
5,2017-12-09,05:21:46,naver,뉴스,6,가상화폐 규제
6,2017-12-09,05:21:46,naver,뉴스,7,뉴스데스크 하차
7,2017-12-09,05:21:46,naver,뉴스,8,일요일 전국 눈이나 비
8,2017-12-09,05:21:46,naver,뉴스,9,전작권 전환 조건 조속히 갖춰야
9,2017-12-09,05:21:46,naver,뉴스,10,예루살렘 수도 인정


In [17]:
x = WebPageUsingBS(Resource()[1], 'html')
x2 = x.find('td', class_='aside')
x3 = x2.find('div', class_='hottopic')
x4 = x3.find_all('ol', class_='type15')

In [11]:
for i in x4:
    print (i.attrs['id'])
    for ii in i.find_all('li'):
        rank = ii.find('em').text
        newstopic = ii.find('a')['title']
        print (rank, newstopic)

newstopic_news
1 박주원 법적대응
2 틸러슨 美대사관 예루살렘 이전
3 서울대병원 파업
4 브렉시트 1단계 협상 타결
5 비결핵항산균 검출
6 가상화폐 규제
7 뉴스데스크 하차
8 일요일 전국 눈이나 비
9 전작권 전환 조건 조속히 갖춰야
10 예루살렘 수도 인정
newstopic_entertain
1 모두의 연애 변우석
2 나 혼자 산다 전현무
3 그것이 알고싶다
4 백조클럽 손연재
5 스켈레톤 윤성빈
6 워너원고 박지훈
7 언터처블 진구
8 언터처블 고준희
9 효리네 민박 이효리
10 강소라 결별


In [181]:
ii.find('a')['title']

'모두의 연애 변우석'

'1'

In [109]:
list_ranking

[]

In [65]:
for i in list_ranking:
    rank = i.find('em')text
    
    print (i)
    break

<li>
<em class="num_news num1">1</em><span class="screen_out">위</span>
<strong class="tit_g"><a class="link_txt" href="https://search.daum.net/search?w=tot&amp;q=%ED%8F%AC%ED%95%AD+%EC%A7%80%EC%A7%84&amp;DA=23H&amp;rtmaxcoll=1TH" target="_blank">포항 지진</a></strong>
<span class="info_status">
<span class="screen_out">순위</span>
<span class="ico_news2 ico_up">468<span class="screen_out">상승</span></span>
</span>
</li>


In [67]:
i.find('em').text

'1'

In [69]:
at = i.find('a')

In [72]:
at.text

'포항 지진'

In [74]:
from datetime import datetime

In [101]:
x = datetime.now()

In [102]:
x.date()

datetime.date(2017, 12, 9)

In [103]:
x.time()

datetime.time(3, 48, 41, 738921)

'2017-12-09'

'03:48:41'